In [60]:
import pandas as pd
import os

In [61]:
df_bz = pd.read_excel("标准映射编码表.xlsx")
df_nz = pd.read_excel("文件名称映射表.xlsx")
name_mapping = df_bz.set_index('字段代码')['字段名称'].to_dict()
nz_mapping = df_nz.set_index('属性表名')['层名称'].to_dict()

In [62]:
df_bz['字段类型'].unique()

array(['Int', 'Char', 'Double', 'Float', 'Doubl e'], dtype=object)

In [63]:
type_mapping = {
    'Integer': 'Int',
    'SmallInteger': 'Short',
    'String': 'Char',
    'Double': 'Double',
    'Single': 'Float'
}

In [66]:
# 设置包含多个 CSV 的文件夹路径
csv_folder = r"shap统计"  # 替换为你的目录路径
output_folder = r"标准映射后"  # 输出 Word 文件目录

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 遍历文件夹中的所有 CSV 文件
for file in os.listdir(csv_folder):
    if file.lower().endswith(".csv"):
        csv_path = os.path.join(csv_folder, file)
        df = pd.read_csv(csv_path, encoding="gbk")
        # 3. 替换FieldName列
        df = df[~df['Type'].isin(['OID', 'Geometry'])]
        df['Filename'] = df['Filename'].apply(lambda x: x[:-4] if x.endswith('.shp') else x)
        df['Filename'] = df['Filename'].replace(nz_mapping)
        df['FieldName'] = df['FieldName'].map(name_mapping)
        df['Type'] = df['Type'].map(type_mapping)
        # 4. 处理未映射的字段（保持原值或设为空）
        df['FieldName'] = df['FieldName'].fillna(df['FieldAlias'])
        # 保存 Word 文档
        output_file = os.path.join(output_folder, os.path.splitext(file)[0] + ".csv")
        df.to_csv(output_file,index=False,encoding="gbk")